In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import math

#Torch-related imports
import torch
import torch.distributions as D
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Function

#Model-specific imports
from SBM_SDE import *
from obs_and_flow import *
from training import calc_log_lik

In [2]:
torch.manual_seed(0)
np.random.seed(0)

In [3]:
temp_ref = 283
temp_rise = 5 #High estimate of 5 celsius temperature rise by 2100. 

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix parameters
c_SOC = 0.1
c_DOC = 0.0001
c_MBC = 0.001
s_SOC = 0.001
s_DOC = 0.001
s_MBC = 0.001

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix parameters
c_SOC = 2
c_DOC = 0.05
c_MBC = 0.1
c_EEC = 0.01
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

#System parameters from deterministic AWB-ECA model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB-ECA diffusion matrix parameters
c_SOC = 2
c_DOC = 0.05
c_MBC = 0.1
c_EEC = 0.01
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_ECA_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [4]:
#Set flow NN parameters.

devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")
dt_flow = 0.2
t = 1000
n_flow = int(t / dt_flow) + 1
t_span = np.linspace(0, t, n_flow)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n_flow, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
l_r = 5e-4
niter = 5501
piter = 250
batch_size = 3 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
obs_error_scale = 0.1 #Proportion of the mean of observation error standard deviation.

x0_SCON = [40, 0.08, 0.8] #Initial condition means for SCON

In [5]:
#Obtain temperature forcing function.
temp_tensor = temp_gen(t_span_tensor, temp_ref, temp_rise)
print(temp_tensor)

#Obtain SOC and DOC pool litter input vectors for use in flow SDE functions.
i_s_tensor = i_s(t_span_tensor) #Exogenous SOC input function
i_d_tensor = i_d(t_span_tensor) #Exogenous DOC input function
print(i_s_tensor)
print(i_d_tensor)

tensor([[[283.0000],
         [283.5248],
         [284.0482],
         ...,
         [281.4880],
         [281.1925],
         [280.9200]]])
tensor([[[0.0010],
         [0.0010],
         [0.0010],
         ...,
         [0.0013],
         [0.0013],
         [0.0013]]])
tensor([[[1.0000e-04],
         [1.0001e-04],
         [1.0001e-04],
         ...,
         [1.3286e-04],
         [1.3286e-04],
         [1.3287e-04]]])


In [6]:
def train(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, SDEFLOW, ObsModel, csv_to_obs_df, DATA_CSV, OBS_ERROR_SCALE, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, TEMP_TENSOR, TEMP_REF, C0, DRIFT_DIFFUSION, PARAMS_DICT): 
    #Read-in observation information. 
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM, T, OBS_ERROR_SCALE)
    obs_means = LowerBound.apply(obs_means, 1e-6)
    #Pass observation information to `ObsModel`.
    obs_model = ObsModel(DEVICE, obs_times, DT, obs_means, obs_error)
    net = SDEFlow(DEVICE, obs_model, STATE_DIM, T, DT, N, I_S_TENSOR, I_D_TENSOR, cond_inputs = 3, num_layers = 6).to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr = L_R)
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e10
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0_tensor = torch.tensor(C0).to(DEVICE) #Convert initial conditions from list to tensor for X0 prior object.
    #C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE)
    PARAMS_DICT_TENSOR = {k: torch.tensor(v).expand(BATCH_SIZE) for k, v in PARAMS_DICT.items()}
    X0_prior = D.normal.Normal(loc = C0_tensor, scale = OBS_ERROR_SCALE * C0_tensor) #Setting prior noise = observation noise for now.
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for i in range(NITER):
            net.train()
            optimizer.zero_grad()
            C_PATH, log_prob = net(BATCH_SIZE) #For obs_and_flow.py
            #C_PATH = torch.cat([C0, C_PATH], 1) #Learning initial conditions in this version. #Append deterministic CON initial conditions conditional on parameter values to C path.
            if i <= PRETRAIN_ITER:
                l1_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l1_norm.item())
                #l2_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                #l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                #best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                #norm_losses.append(l2_norm.item())
                if i % 10 == 0:
                    ma_norm_loss = sum(norm_losses[-10:]) / len(norm_losses[-10:])
                    print(f"\nMoving average norm loss at {iter} iterations is: {ma_norm_loss}. Best norm loss value is: {best_loss_norm}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\nC_PATH =', C_PATH)
                l1_norm.backward()
                #l2_norm.backward()
            else:
                log_lik = calc_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), DT, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE), TEMP_TENSOR.to(DEVICE), TEMP_REF, DRIFT_DIFFUSION, PARAMS_DICT)
                neg_ELBO = -X0_prior.log_prob(C_PATH[:, 0, :]).sum(-1).mean() - log_lik.mean() - obs_model(C_PATH, PARAMS_DICT_TENSOR) + log_prob.mean()
                best_loss_ELBO = neg_ELBO if neg_ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(neg_ELBO.item())
                if i % 10 == 0:             
                    ma_elbo_loss = sum(ELBO_losses[-10:]) / len(ELBO_losses[-10:])
                    print(f"\nMoving average ELBO loss at {iter} iterations is: {ma_elbo_loss}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\nC_PATH =', C_PATH)
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if i % 100000 == 0 and i > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()
    return net, ELBO_losses, norm_losses

In [ ]:
net, ELBO_losses, norm_losses = train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'y_from_x_t_1000_dt_0-01.csv', obs_error_scale, state_dim_SCON, t, dt_flow, n_flow, t_span_tensor, i_s_tensor, i_d_tensor, temp_tensor, temp_ref, x0_SCON, drift_diffusion_SCON_C, SCON_C_params_dict)


Train Diffusion:   0%|          | 0/5501 [00:00<?, ?it/s]


Moving average norm loss at <built-in function iter> iterations is: 9000048777.76875. Best norm loss value is: 487777.6875.

C_PATH mean = tensor([[0.8412, 0.8549, 0.8230],
        [0.8406, 0.8506, 0.8430],
        [0.8338, 0.8452, 0.8374]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[0.7979, 0.4989, 0.8650],
         [1.0883, 0.3769, 2.7311],
         [0.3203, 0.7267, 1.9741],
         ...,
         [0.5538, 0.7271, 0.4697],
         [0.6251, 1.0343, 0.5232],
         [0.5830, 1.1936, 0.6196]],

        [[0.7932, 0.9848, 0.8966],
         [0.2745, 1.7503, 1.1032],
         [0.2097, 0.5167, 1.5775],
         ...,
         [0.7713, 1.1644, 0.7867],
         [0.6278, 0.5162, 0.7906],
         [0.8528, 0.6983, 0.7032]],

        [[0.5078, 1.8464, 0.8937],
         [1.3542, 0.9721, 0.2880],
         [3.1418, 1.7542, 0.5738],
         ...,
         [1.0607, 0.4409, 0.7835],
         [1.4567, 0.4050, 0.8300],
         [1.0807, 0.6015, 0.7105]]], grad_fn=<AddBackward0>)



Train Diffusion:   0%|          | 10/5501 [01:22<12:02:09,  7.89s/it]


Moving average norm loss at <built-in function iter> iterations is: 484292.625. Best norm loss value is: 481173.78125.

C_PATH mean = tensor([[1.1543, 0.9091, 1.0381],
        [1.1465, 0.9087, 1.0403],
        [1.1530, 0.9061, 1.0468]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[0.6522, 1.1212, 1.3921],
         [1.4355, 1.9493, 1.7745],
         [2.1817, 1.5417, 1.7370],
         ...,
         [3.0407, 0.2333, 0.4067],
         [0.8372, 0.9192, 0.8916],
         [1.6845, 0.8087, 0.8938]],

        [[0.7257, 1.5692, 1.2102],
         [1.2684, 1.1243, 1.5937],
         [1.3636, 0.9959, 1.1029],
         ...,
         [0.9383, 1.1271, 0.9637],
         [2.9841, 0.6358, 1.5744],
         [0.6144, 0.8518, 0.9835]],

        [[0.7725, 1.9250, 1.0835],
         [0.9704, 1.2086, 1.1118],
         [1.0455, 1.3292, 1.2594],
         ...,
         [0.8271, 0.9082, 1.0051],
         [0.9568, 0.5508, 0.8797],
         [1.0169, 0.9112, 1.2045]]], grad_fn=<AddBackward0>)



Train Diffusion:   0%|          | 20/5501 [02:43<13:13:10,  8.68s/it]

...


Train Diffusion:   5%|▌         | 301/5501 [35:28<4:08:11,  2.86s/it]


Moving average ELBO loss at <built-in function iter> iterations is: 1067581267.2. Best ELBO loss value is: 582595072.0.

C_PATH mean = tensor([[18.9840,  0.9058,  5.1434],
        [18.8174,  0.9826,  5.1329],
        [18.9860,  0.9473,  5.1385]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[ 7.7829, 12.7937,  1.1590],
         [ 8.1005,  0.6705,  3.4888],
         [30.2208,  0.9365,  8.3738],
         ...,
         [ 7.9470,  0.1963,  6.2992],
         [10.9562,  0.4556,  0.8719],
         [15.7282,  2.1421,  7.2462]],

        [[ 9.0329, 13.4010,  0.6163],
         [ 8.7897,  0.9871,  1.2650],
         [ 8.5882,  0.1210,  5.1321],
         ...,
         [ 5.9520,  0.3728,  2.0699],
         [22.8261,  0.8695,  3.8424],
         [ 7.1768,  1.3411,  2.8852]],

        [[12.4035,  7.4203,  0.1260],
         [28.7906,  0.1178,  8.7194],
         [ 7.9396,  0.4704,  5.6070],
         ...,
         [25.5294,  0.4931,  3.8338],
         [ 8.2300,  0.4894,  2.3584],
         [29.4447,  0.243


Train Diffusion:   6%|▌         | 311/5501 [35:55<4:02:01,  2.80s/it]


Moving average ELBO loss at <built-in function iter> iterations is: 1098671302.4. Best ELBO loss value is: 582595072.0.

C_PATH mean = tensor([[18.7936,  0.9819,  5.1542],
        [18.7359,  0.9460,  5.1916],
        [18.9625,  0.9578,  5.1665]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[8.0124e+00, 1.2884e+01, 3.1176e-01],
         [6.3595e+00, 2.6022e+00, 1.0657e+00],
         [3.9470e+01, 1.1032e-03, 2.1633e+00],
         ...,
         [6.9539e+00, 3.7213e-01, 2.2249e+00],
         [2.2996e+01, 6.6196e-01, 4.1795e+00],
         [7.3590e+00, 1.7491e+00, 2.7588e+00]],

        [[1.2461e+01, 7.8244e+00, 1.4155e-01],
         [2.8733e+01, 8.9610e-02, 7.0100e+00],
         [3.9697e+00, 1.0864e+00, 2.4718e+01],
         ...,
         [2.8286e+00, 3.6551e+00, 6.3459e+00],
         [4.2538e+00, 2.8032e-03, 1.8747e+00],
         [2.5961e+01, 1.6482e-02, 7.7256e+00]],

        [[8.7907e+00, 1.3261e+01, 2.3117e+00],
         [1.8928e-01, 1.4160e+00, 1.9823e+01],
         [2.3439e-05, 4.045


Train Diffusion:   6%|▌         | 320/5501 [36:22<4:08:44,  2.88s/it]